In [1]:
# multiprocess data from filename pool

from mptracker import MPTracker,TiffFileSequence
import cv2
from tifffile import imread
cv2.setNumThreads(0)

from json import load as jload
import  numpy as np
filename = '/home/joao/data/eyecam/181112_JC086_2P_JC/run00_gray/20181112_run000_00000000.tif'
parameterfile = '/home/joao/data/eyecam/181112_JC086_2P_JC/run00_gray/20181112_run000.json'
seq = TiffFileSequence(filename)

with open(parameterfile,'r') as fd:
    trackerpar = jload(fd)
mptracker = MPTracker(trackerpar)

def process_tiff(fname,parameters):
    cv2.setNumThreads(0)
    tracker = MPTracker(parameters)
    dat = imread(fname)
    if len(dat.shape) < 3:
        dat = [dat]
    res = []
    for frame in dat:
        res.append(tracker.apply(frame))
    del tracker
    return res

from multiprocess import Pool
from functools import partial
import time
nprocesses = 12
ts = time.time()
with Pool(nprocesses) as pool:
    res = pool.map(partial(process_tiff,parameters = trackerpar),seq.filenames)
    res = np.vstack(res)
    
toc = time.time() - ts
print('Analysed {0} frames in {1} s [{2} fps]'.format(seq.nFrames,toc,seq.nFrames/toc))

/home/joao/data/eyecam/181112_JC086_2P_JC/run00_gray
72
There are 18229 frames in 72 files
Analysed 18229 frames in 6.704983711242676 s [2718.7239798113555 fps]


In [80]:
# With enhanced
class playStack(object):
    gaussBlurSize = 3
    gaussBlurSigma = 2
    clahe = cv2.createCLAHE(clipLimit=35.0, tileGridSize=(8,8))
    paused = False
    def __init__(self,data,res):
        self.data = data
        self.res = res
        self.N = data.nFrames
        self.iFrame = 0
        self.lastImg = self.enhanceImage(self.data.get(self.iFrame))
    def enhanceImage(self,img):
        return self.clahe.apply(cv2.GaussianBlur(img,
                                                 (self.gaussBlurSize,
                                                  self.gaussBlurSize),
                                                 self.gaussBlurSigma))   
    def play(self):
        while True:
            img = self.enhanceImage(self.data.get(self.iFrame)) 
            self.lastImg = img.copy()
            cv2.putText(img, 'frame {0}'.format(self.iFrame), (50,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1.0, (255, 255, 255), 
                        lineType=cv2.LINE_AA)
            if not np.isnan(res[self.iFrame,1][0]):
                xy = (int(res[self.iFrame,1][0]),int(res[self.iFrame,1][1]))
                img = getEllipseMask(img.shape,[xy[0],xy[1]],[res[self.iFrame,3][0]/2,
                                     res[self.iFrame,3][1]/2],[res[self.iFrame,3][2]])
                c = ellipseToContour([xy[1],xy[0]],
                                     res[self.iFrame,3][0]/2.,
                                     res[self.iFrame,3][1]/2.,
                                     0)#res[self.iFrame,3][2]+90)
                img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                cv2.drawContours(img,[c],0,[0,0,255],1)
            img = cv2.circle(img,xy , 2, 255, 1)
            cv2.imshow('img',img)
            if not self.paused:
                self.iFrame = np.mod(self.iFrame + 1,self.N)
            k = cv2.waitKey(3)
            if k == ord('q'):
                break
            elif k == ord('s'):
                self.paused = not self.paused
        cv2.destroyAllWindows()

splay = playStack(seq,res)
splay.play()
#splay.save('/home/joao/data/facecam/whisker_top_view/topview_example.avi')

(105.11105346679688, 119.55891418457031, 32.00214385986328)

In [16]:
# plot overlay
class playStack(object):
    gaussBlurSize = 3
    gaussBlurSigma = 2
    clahe = cv2.createCLAHE(clipLimit=35.0, tileGridSize=(8,8))
    paused = False
    def __init__(self,data,res):
        self.data = data
        self.res = res
        self.N = data.nFrames
        self.iFrame = 0
        self.lastImg = self.enhanceImage(self.data.get(self.iFrame))
    def enhanceImage(self,img):
        return self.clahe.apply(cv2.GaussianBlur(img,
                                                 (self.gaussBlurSize,
                                                  self.gaussBlurSize),
                                                 self.gaussBlurSigma))   
    def play(self):
        while True:
            img = self.enhanceImage(self.data.get(self.iFrame)) 
            self.lastImg = img.copy()
            cv2.putText(img, 'frame {0}'.format(self.iFrame), (50,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1.0, (255, 255, 255), 
                        lineType=cv2.LINE_AA)
            if not np.isnan(res[self.iFrame,1][0]):
                xy = (int(res[self.iFrame,1][0]),int(res[self.iFrame,1][1]))
            
                c = ellipseToContour([xy[1],xy[0]],
                                     res[self.iFrame,3][0]/2.,
                                     res[self.iFrame,3][1]/2.,
                                     0)#res[self.iFrame,3][2]+90)

                cv2.drawContours(img,[c],0,255,1)
            img = cv2.circle(img,xy , 2, 255, 1)
            cv2.imshow('img',img)
            if not self.paused:
                self.iFrame = np.mod(self.iFrame + 1,self.N)
            k = cv2.waitKey(3)
            if k == ord('q'):
                break
            elif k == ord('s'):
                self.paused = not self.paused
        cv2.destroyAllWindows()

splay = playStack(seq,res)
splay.play()
#splay.save('/home/joao/data/facecam/whisker_top_view/topview_example.avi')

In [3]:
from mptracker import ellipseToContour
import pylab as plt
%matplotlib qt
c = ellipseToContour(res[0,1],res[0,3][0]/2,res[0,3][1]/2,res[0,3][2])
plt.imshow(seq.get(0)**0.5,cmap = 'gray')
plt.plot(c[:,0,1],c[:,0,0],'y')

In [59]:
from mptracker import convertPixelToEyeCoords
az,el,theta = convertPixelToEyeCoords(np.hstack(res[1]),
                                      (trackerpar['points'][0],trackerpar['points'][2]),
                                      np.hstack(res[0]))

IndexError: too many indices for array

In [6]:
from mptracker import getEllipseMask
MPTracker.getAugmented??